<a href="https://colab.research.google.com/github/scskalicky/creative_naming_lsa/blob/main/creative_names_cogsci.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Update spaCy to 3.x and manually download 3.2.0 large english model
!pip install spacy --upgrade
!pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_lg-3.2.0/en_core_web_lg-3.2.0.tar.gz

In [2]:
import spacy
print(spacy.__version__)

3.2.1


In [ ]:
# The Google Universal Sentence Encoder for spaCy
# https://github.com/MartinoMensio/spacy-universal-sentence-encoder
# https://stackoverflow.com/questions/52113939/spacy-strange-similarity-between-two-sentences
!pip install git+https://github.com/MartinoMensio/spacy-universal-sentence-encoder.git

In [ ]:
# Download the large USE model. 
!pip install https://github.com/MartinoMensio/spacy-universal-sentence-encoder/releases/download/v0.4.3/xx_use_lg-0.4.3.tar.gz#xx_use_lg-0.4.3

In [ ]:
# RESTART THE RUNTIME AFTER PREVIOUS CELL AND BEFORE NEXT CELL

In [1]:
# import and initliaze nlp to be the USE model
import spacy
import spacy_universal_sentence_encoder
nlp = spacy_universal_sentence_encoder.load_model('en_use_lg')
nlp.add_pipe('universal_sentence_encoder')

Downloaded https://tfhub.dev/google/universal-sentence-encoder-large/5, Total size: 577.10MB



In [4]:
a = 'air ball'
b = 'going to the moon bag'

nlp(a).similarity(nlp(b))

# should be: 
#0.26939963742482953

0.26939963742482953

In [29]:
import pandas as pd
dat = pd.read_csv('/content/drive/MyDrive/creative_naming/creative_naming_raw_answers.csv')

print(len(dat), len(dat[dat['on.task'] == 0]))

1296 53


In [30]:
dat = dat[dat['on.task'] == 1]
dat.columns
len(dat)

1243

In [9]:
nes_dat = dat[dat['group'] == 'NES']
rom_dat = dat[dat['group'] == 'ROM']

In [10]:
nes_answers = nes_dat[['cleaned_answer', 'item']]
rom_answers = rom_dat[['cleaned_answer', 'item']]

In [11]:
items = list(dat['item'].unique())
items

['balloon',
 'box',
 'carrot',
 'fork',
 'shovel',
 'toilet',
 'couch',
 'glasses',
 'wallet']

In [12]:
nes_answers[nes_answers['item'] == 'balloon']['cleaned_answer'].unique()

array(['air ball', 'rubber helium holder', 'poke popper', 'party bubble',
       'air holder', 'party floater', 'helium globe', 'air toy',
       'party popper', 'toy sac air holder', 'floating ball',
       'gas capsule', 'pop and scare', 'air bomb', 'rubber floater',
       'air floater', 'floating object', 'big plastic colored object',
       'loud popper', 'floating sphere', 'air pack',
       'green air container', 'floating ball of air', 'air container',
       'poppable air container', 'air filler', 'green bubble',
       'green rubber float', 'colorful air holder', 'blown air',
       'inflated decoration', 'blimp bird', 'inflatable rubber',
       'air bag', 'inflated elation', 'decorative rubber float',
       'air filled rubber', 'inflatable bauble', 'chocolate shape maker',
       'breath capturer', 'green helium voice changer', 'rubber blow up',
       'floating air', 'bag of air', 'poppable', 'air trapper',
       'elastic air holder'], dtype=object)

In [ ]:
# Create similarity scores 
import time
from collections import defaultdict

def sim_dict(answers, items):
  
  output_dict = defaultdict(dict)
  for item in items:
    item_answers = list(answers[answers['item'] == item]['cleaned_answer'])
    print(f'computing similarity for items in {item}')
    #print(item_answers)
    for index, answer in enumerate(item_answers):
      answer_sims_dict = dict()
      print(f'computing similarity for {answer}...')
      print(f'{index+1} of {len(item_answers)}')
      answer_sims = [(x, nlp(answer).similarity(nlp(x))) for x in item_answers]
      output_dict[item][answer] = dict(answer_sims)
      #print(len(output_dict[item].keys()))

  return output_dict

In [ ]:
# takes a few minutes to calculate
nes_sims = sim_dict(nes_answers, items)

In [ ]:
len(list(nes_sims['balloon'].keys()))

47

In [ ]:
import pickle
afile = open(r'/content/drive/MyDrive/creative_naming/nes_sims.pkl', 'wb')
pickle.dump(nes_sims, afile)
afile.close()

In [ ]:
rom_sims = sim_dict(rom_answers,items)

In [ ]:
rom_sims['balloon']

In [ ]:
bfile = open(r'/content/drive/MyDrive/creative_naming/rom_sims.pkl', 'wb')
pickle.dump(rom_sims, bfile)
bfile.close()

In [31]:
# calculate similarity between answers and the name of the object

# the list comprehension is probably 100x faster than the loop;
#for index, row in dat.iterrows():
 # dat.loc[index, 'sim_to_name'] = nlp(dat.loc[index, 'cleaned_answer']).similarity(nlp(dat.loc[index, 'item']))
result = [nlp(x).similarity(nlp(y)) for x, y in zip(dat['cleaned_answer'], dat['item'])]
dat['sim_to_item'] = result

In [32]:
dat

,subject,group,item,original_answer,cleaned_answer,on.task,sim_to_item
53,49998,NES,balloon,air ball /,air ball,1,0.630809
54,49998,NES,box,Compressed paper container,compressed paper container,1,0.244441
55,49998,NES,carrot,orange veggie,orange veggie,1,0.617006
56,49998,NES,fork,eating tool,eating tool,1,0.414731
57,49998,NES,shovel,digger,digger,1,0.623323
...,...,...,...,...,...,...,...
1291,R_Zres5zjqgT36pEt,ROM,glasses,clarifier,clarifier,1,0.217890
1292,R_Zres5zjqgT36pEt,ROM,shovel,plant remover,plant remover,1,0.421628
1293,R_Zres5zjqgT36pEt,ROM,toilet,pee-pee taker,pee pee taker,1,0.368440
1294,R_Zres5zjqgT36pEt,ROM,wallet,money collector,money collector,1,0.441208


In [36]:
dat.to_csv('/content/drive/MyDrive/creative_naming/creative_naming_raw_answers_withSIM.csv')

In [33]:
print(spacy.__version__)

3.0.7


In [34]:
from platform import python_version
print(python_version())

3.7.12
